In [1]:
import sys
sys.path.insert(0, "extern/levanter/src")

from transformers import RobertaConfig as HFRobertaConfig
from transformers import RobertaForMaskedLM as HFRobertaForMaskedLM
from extern.levanter.src.levanter.models.roberta import RobertaConfig
from extern.levanter.src.levanter.models.roberta import RobertaForMaskedLM

import jax
import jax.random as jrandom
import jax.numpy as jnp
import haliax as hax

c:\Users\julie\anaconda3\envs\levanter2\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2025-05-22 17:30:14,345	INFO util.py:154 -- Missing packages: ['ipywidgets']. Run `pip install -U ipywidgets`, then restart the notebook server for rich notebook output.


In [2]:
import os

print(os.getcwd())

c:\My_Documents\bigbert


In [3]:
def load_weights_from_hf():
    # Load the Hugging Face model
    hf_model = HFRobertaForMaskedLM.from_pretrained("roberta-base")
    hf_config = HFRobertaConfig.from_pretrained("roberta-base")

    hf_config.hidden_dropout_prob = 0
    hf_config.attention_probs_dropout_prob = 0
    # hf_config.pad_token_id = -1

    lv_config = RobertaConfig.from_hf_config(hf_config)

    converter = lv_config.hf_checkpoint_converter()

    model = converter.load_pretrained(
        lv_config.model_type,
        lv_config,
        axis_mapping=None, 
        dtype="float32",  
    )

    print(converter.Vocab)
    
    #print("Weights loaded successfully.")
    return model, lv_config, hf_model, hf_config

lv_model, lv_config, hf_model, hf_config = load_weights_from_hf()


Loading weights: 100%|██████████| 203/203 [00:10<00:00, 19.33it/s]


vocab(50265)


c:\Users\julie\anaconda3\envs\levanter2\lib\site-packages\haliax\partitioning.py:116: RuntimeWarning: No resource mapping found. Not sharding.
  warnings.warn("No resource mapping found. Not sharding.", RuntimeWarning)


In [4]:
print(lv_config.vocab_size)
print(hf_config.vocab_size)

print(lv_config.max_position_embeddings)
print(hf_config.max_position_embeddings)

50265
50265
514
514


In [5]:
import torch
import numpy as np

In [6]:
def named_to_tensor(named_array):
    out_tensor = torch.tensor(np.array(named_array.array))
    return out_tensor

def tensor_to_named(in_tensor, axes):
    named_array = hax.NamedArray(np.array(in_tensor), axes)
    return named_array

In [7]:
# Compare outputs
def check(my_out, hf_out, precision=1e-4):
    acc = np.isclose(hf_out, my_out, rtol=precision, atol=precision).mean()
    diff = np.abs(my_out - hf_out).mean()
    return f"Accuracy: {acc:.4f}, Avg Difference: {diff:.6f}"

In [8]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("roberta-base")

In [9]:
Batch = hax.Axis("batch", 1)
Pos = lv_config.Pos
KeyPos = lv_config.KeyPos
Vocab = hax.Axis("vocab", tokenizer.vocab_size)

key = jrandom.PRNGKey(42)
key_var, key_model = jrandom.split(key, 2)

In [15]:
# prompt = tokenizer("Mark <mask> is the CEO of Facebook, located in <mask> <mask>, California.", return_tensors="pt", padding='max_length', max_length=514)
prompt = tokenizer("Paris is the <mask> of France.", return_tensors="pt", padding='max_length', max_length=514)
# prompt = tokenizer("Mark Zuckerberg is the boss of Facebook, located in Palo Alto, California.", return_tensors="pt", padding='max_length', max_length=514)

In [16]:
lv_prompt = {"input_ids": tensor_to_named(prompt["input_ids"], (Batch, Pos)), "attention_mask": tensor_to_named(prompt["attention_mask"], (Batch, KeyPos))}

In [17]:
# prompt["input_ids"]

In [18]:
# attention_mask_tensor = torch.ones(size = (Batch.size, Pos.size), dtype = int)
# prompt["attention_mask"] = attention_mask_tensor

In [19]:
# lv_prompt = {k: hax.NamedArray(np.array((prompt[k])), axes = (Batch, Pos)) for k in prompt.keys()}

In [20]:
# input_ids = hax.random.randint(key_var, shape = (Batch, Pos), minval = lv_config.eos_token_id+1, maxval = lv_config.vocab_size)
# attention_mask = hax.ones(shape = (Batch, Pos), dtype=int)

# input_ids_tensor = named_to_tensor(input_ids)
# attention_mask_tensor = named_to_tensor(attention_mask)

In [21]:
# input_ids = haliax.random.randint(jax.random.PRNGKey(42), shape = (haliax.Axis("batch", 1), haliax.Axis("position", 512)), minval = 4, maxval = 50265)
# input_ids = torch.ones((1, 512), dtype=int) * 100

In [22]:
# input_ids_tensor = torch.randint(low = 5, high = 50265, size = 514)

In [23]:
# input_ids_tensor = torch.randint(low = lv_config.eos_token_id+1, high = lv_config.vocab_size, size = (Batch.size, Pos.size))
# # input_ids_tensor[:, 0] = 0
# # input_ids_tensor[:, (lv_config.max_position_embeddings // 2):] = 1
# # input_ids_tensor[:, (lv_config.max_position_embeddings // 2)] = 2

# # # input_ids_tensor = torch.ones((Batch.size, Pos.size), dtype=int) * 100

# idx = torch.randint(low = 1, high=lv_config.max_position_embeddings, size = (1,))
# # idx = lv_config.max_position_embeddings // 2

# attention_mask_tensor = torch.ones(size = (Batch.size, KeyPos.size), dtype = int)
# # attention_mask_tensor[:, idx:] = 0

# attention_mask = tensor_to_named(attention_mask_tensor, (Batch, KeyPos))

# # # attention_mask_tensor = torch.ones(size = (Batch.size, Pos.size), dtype = int)
# # # attention_mask_tensor[:, idx:] = 0

# # # attention_mask = tensor_to_named(attention_mask_tensor, (Batch, KeyPos))

# input_ids = tensor_to_named(input_ids_tensor, (Batch, Pos))
# prompt = {"input_ids": input_ids_tensor, "attention_mask": attention_mask_tensor}
# lv_prompt = {"input_ids": input_ids, "attention_mask": attention_mask}

# # # prompt = {"input_ids": input_ids_tensor}
# # # lv_prompt = {"input_ids": input_ids}

In [24]:
# prompt = {"input_ids": input_ids_tensor, "attention_mask": attention_mask_tensor}
# lv_prompt = {"input_ids": input_ids, "attention_mask": attention_mask}

In [25]:
# def create_position_ids_from_input_ids(input_ids, past_key_values_length=0):
#     mask = hax.not_equal(input_ids, lv_config.pad_token_id) * 1
#     incremental_indices = (hax.cumsum(mask, axis=lv_config.Pos).astype(mask) + past_key_values_length) * mask
#     incremental_indices -= mask.all(axis=Pos)
#     return incremental_indices + lv_config.pad_token_id

# def create_position_ids_from_input_ids(input_ids, past_key_values_length=0):
#     return hax.arange(axis = Pos, start = 0, dtype=jnp.int32)

def create_position_ids_from_input_ids(input_ids, PosInput, past_key_values_length=0):
    mask = hax.not_equal(input_ids, lv_config.pad_token_id) * 1
    incremental_indices = (hax.cumsum(mask, axis=PosInput).astype(mask) + past_key_values_length) * mask
    incremental_indices -= mask.all(axis=PosInput) * lv_config.pad_token_id
    return incremental_indices

In [26]:
# dict = hf_model.state_dict()
# print(dict["roberta.embeddings.position_embeddings.weight"].shape)
# print(dict["roberta.embeddings.position_embeddings.weight"])

In [27]:
position_ids = create_position_ids_from_input_ids(lv_prompt["input_ids"], Pos)

lv_prompt["position_ids"] = position_ids
prompt["position_ids"] = torch.from_numpy(np.array(position_ids.array))

In [28]:
print(check(np.array(lv_prompt["input_ids"].array), np.array(prompt["input_ids"])))
# print(check(np.array(lv_prompt["attention_mask"].array), np.array(prompt["attention_mask"])))
# print(check(np.array(lv_prompt["position_ids"].array), np.array(prompt["position_ids"])))

Accuracy: 1.0000, Avg Difference: 0.000000


In [ ]:
# input = torch.tensor([[    0,   287,     5,  8603, 13802, 50264,   109,    45,  5152,
#            6,     5,  2853,   364, 22448,  1790,     9,     5,  7155,
#          811,  2938, 14958,     5,  4286, 50264, 27099,  3069, 34337,
#           11,   645,     7,  2142,   106,    15, 10779,    14,    74,
#         3680,   146,  7155,   493,  2217, 50264,    11,     5,   997,
#            4,   616,    23,   498, 50264,  1364,     7, 50264,  2093,
#            6,   215,    25,    10, 50264,  5853, 29471, 50264, 50264,
#         4284,     6,    97,  3365,  1303,  1402,   453, 50264,     5,
#        41977,  1187,   372, 13250,     4,   509,   215,   919,     6,
#        50264,  7150,     6,  3374,    98, 33415,    14,    37,  4091,
#        50264,  1790,    39, 50264, 50264, 23487,    88,     5,  5546,
#            9,  2912,   424,  1571, 50264,     6,  7391,     7,     5,
#         5631,     9,   211,  9636,  7305, 50264,  1850, 50264,    49,
#          884,     6, 25000,  1180,     4,   497,     5,   276,    86,
#            6,  4785,   624,  7155,   811,  2938,  9539,   517,     7,
#        24300,     5,  8603, 13802,    11,   645,     7,  1744, 50264,
#          308,  3168,     4,   289,  9968,    30,   258,  4181,     8,
#        11058,     6,     8, 50264, 50264,     5,  2621,     9,    10,
#        40048,   624,    49,  5546, 50264, 50264, 41977,  1187, 12420,
#          517,     7,   489,  1235,  4299,   150,    23,     5,   276,
#           86,  1032,     7, 50264,     5,  7155,   811,   997,  1351,
#            4,   152,  1388, 50264,     5,  8603, 13802,    18, 20510,
#         1036,     6, 14658,  6466,  1694,     6,    54,    56,    57,
#         1682, 50264, 50264,  2237,     6,    16, 50264,     7,     5,
#          812,   343,     9, 10714,   571, 21645,    11,   645,     7,
#         1455,  1283, 31630,  1295,     5, 50264,  3878,     8, 13595,
#            5,   588, 40048,     6,     5,  7155,   811,  1292,    14,
#           56,  1238, 12255, 50264,  6213, 12231,     4, 50118, 50264,
#            0, 50264, 50264, 47408, 50118, 50264, 50264, 50264, 18089,
#        45994, 50118,     2,     0,  1596, 38996, 50264,    58,  2622,
#            6,   511, 50264,     9,     5,   177,    18,  1049,  2182,
#        37813,  5902,  3245,     8,   248, 50264,   102,     4,   252,
#           58,  2211,   267, 38183,   117,   468, 44068,  6374,   155,
#           35,  8603,   139,   295,  7387,   732,   967,  1439, 50264,
#          289,  1113,    36, 49902, 42393, 21402, 20024, 48018, 50033,
#        49080, 49587, 49432, 48947, 49017, 50264, 47111, 16948,  8384,
#        48885, 48247, 49045, 48537,  9085, 47780, 48018, 36484, 50264,
#            6, 50264,     4,   468, 44068,  6374, 50264,     5, 36954,
#          155,    35,    20, 27381,     9, 50264,  8603, 13802, 41080,
#          238, 50264,    30, 50264,  2160, 16819, 20483,   354,  6498,
#            8, 50264,   703,    11, 50264,  7267, 50264, 50264, 50264,
#         1538, 50264, 50264, 20526, 50264,  3066,  2678,   227,  1466,
#            8,  1125,   131,     8,  2211,   267, 38183,   117, 50264,
#        44068,  6374,   155,    35,   111, 50264,  7387,  1890,  1794,
#          118,   117, 20270, 13253, 10233, 50264,    12,    36, 49902,
#        42393, 21402, 20024, 48018, 50033, 49080, 49587, 50264, 48947,
#        49017, 50264,   111, 36484,  8906, 10470, 49045, 48583, 50264,
#        49126, 10809, 50264, 49902, 48732,    27, 49583, 50264,  6474,
#            4,   468, 44068,  6374,     9,     5, 36954,   155,   111,
#          133, 50264,     9,     5, 50264, 50264,   238, 22827,    30,
#        14688, 16819,  7303, 50264,     8,  2140,   703,    11,    10,
#          881,  3149, 50264, 10912,  1638,  6498, 20796,   225, 50264,
#         1125,     4, 50118,     2,     0, 45994,  8911, 45994, 50118,
#        50264,     0,   111, 10567,   256,  2482,  5982, 20681,     6,
#          644,  1105,     6, 44023, 50118,     2,     0,    20,  4046,
#         2751,  4152,    21,  8967,    11, 39947,    25,    41, 50264,
#         2726,   225,   337,     9,  8068,    60, 50264,    14,    24,
#           21]])
# print(input)

tensor([[    0,   287,     5,  8603, 13802, 50264,   109,    45,  5152,     6,
             5,  2853,   364, 22448,  1790,     9,     5,  7155,   811,  2938,
         14958,     5,  4286, 50264, 27099,  3069, 34337,    11,   645,     7,
          2142,   106,    15, 10779,    14,    74,  3680,   146,  7155,   493,
          2217, 50264,    11,     5,   997,     4,   616,    23,   498, 50264,
          1364,     7, 50264,  2093,     6,   215,    25,    10, 50264,  5853,
         29471, 50264, 50264,  4284,     6,    97,  3365,  1303,  1402,   453,
         50264,     5, 41977,  1187,   372, 13250,     4,   509,   215,   919,
             6, 50264,  7150,     6,  3374,    98, 33415,    14,    37,  4091,
         50264,  1790,    39, 50264, 50264, 23487,    88,     5,  5546,     9,
          2912,   424,  1571, 50264,     6,  7391,     7,     5,  5631,     9,
           211,  9636,  7305, 50264,  1850, 50264,    49,   884,     6, 25000,
          1180,     4,   497,     5,   276,    86,  

In [30]:
# def check_dicts(my_dict, hf_dict):
#     print(my_dict.keys())
#     print(hf_dict.keys())

#     my_keys = set(my_dict)
#     hf_keys = set(hf_dict)

#     both = hf_keys.union(my_keys)

#     correct_msg = "Accuracy: 1.0000, Avg Difference: 0.000000"

#     for k in both:
#         if k not in my_keys:
#             print(f"Key {k} not in my_keys!")
#             continue

#         if k not in hf_keys:
#             print(f"Key {k} not in hf_keys!")
#             continue
        
#         check_str = check(my_dict[k], hf_dict[k])
#         if check_str != correct_msg:
#             print(check_str)

# my_dict = lv_model.to_state_dict()
# hf_dict = hf_model.state_dict()

# my_dict = {k: np.array(my_dict[k]) for k in my_dict.keys()}
# hf_dict = {k: np.array(hf_dict[k]) for k in hf_dict.keys()}

# check_dicts(my_dict, hf_dict)

# print(check(my_dict["lm_head.decoder.bias"], hf_dict["lm_head.decoder.bias"]))
# print(check(my_dict["lm_head.decoder.bias"], hf_dict["lm_head.bias"]))
# print(check(hf_dict["lm_head.decoder.bias"], hf_dict["lm_head.bias"]))

In [31]:
print(hf_config.bos_token_id)
print(hf_config.pad_token_id)
print(hf_config.eos_token_id)

0
1
2


In [32]:
# word_embeddings = hf_dict["roberta.embeddings.word_embeddings.weight"]
# input_ids = lv_prompt["input_ids"]
# input_ids_np = np.array(lv_prompt["input_ids"].array)

# input_embeds_np = word_embeddings[input_ids_np]
# input_embeds_hax = hax.NamedArray(input_embeds_np, axes=(Batch, Pos, lv_config.Embed))

# cond = (input_ids == lv_config.pad_token_id)
# cond = hax.broadcast_to(cond, input_embeds_hax.axes)

# input_embeds_hax = hax.where(
#     cond,
#     hax.zeros_like(input_embeds_hax),
#     input_embeds_hax
# )

# input_embeds_torch = torch.from_numpy(np.array(input_embeds_hax.array))
# new_lv_prompt = dict()
# new_hf_prompt = dict()

# new_lv_prompt["input_embeds"] = input_embeds_hax
# new_hf_prompt["inputs_embeds"] = input_embeds_torch

# new_lv_prompt["attention_mask"] = lv_prompt["attention_mask"]
# new_hf_prompt["attention_mask"] = prompt["attention_mask"]

# new_lv_prompt["position_ids"] = lv_prompt["position_ids"]
# new_hf_prompt["position_ids"] = prompt["position_ids"]

In [33]:
# batch = hax.Axis("batch", 1)
# heads = hax.Axis("heads", 12)
# position = hax.Axis("position", 514)
# key_position = hax.Axis("key_position", 514)

# attention_scores = hax.ones({batch: batch.size, heads: heads.size, position: position.size, key_position: key_position.size})

# attention_mask = lv_prompt["attention_mask"]

# attention_mask = (attention_mask == 0) * -1e12

# attention_mask = attention_mask.rename({"position": "key_position"})
# # print(attention_mask)
# print(attention_mask.axes)

# attention = attention_scores + attention_mask

# print(attention)

In [34]:
lv_result = lv_model(**lv_prompt)
hf_result = hf_model(**prompt)

# lv_result = lv_model(**new_lv_prompt)
# hf_result = hf_model(**new_hf_prompt)

In [35]:
lv_result_logits = torch.from_numpy(np.array(lv_result.array))
hf_result_logits = hf_result.logits

In [36]:
print(f"lv_logits vs hf_logits: {check(np.array(lv_result_logits), np.array(hf_result_logits.detach()))}")
print(f"lv_tokens vs hf_tokens: {check(np.array(lv_result_logits.argmax(dim=-1)), np.array(hf_result_logits.argmax(dim=-1).detach()))}")
print(check(np.array(lv_result_logits.argmax(dim=-1)), np.array(prompt["input_ids"])))
print(check(np.array(hf_result_logits.argmax(dim=-1).detach()), np.array(prompt["input_ids"])))

lv_logits vs hf_logits: Accuracy: 1.0000, Avg Difference: 0.000005
lv_tokens vs hf_tokens: Accuracy: 1.0000, Avg Difference: 0.000000
Accuracy: 0.0117, Avg Difference: 158.542802
Accuracy: 0.0117, Avg Difference: 158.542802


In [37]:
# from levanter.models.lm_model import MaskedLmExample

# example = MaskedLmExample.masked_lm(tokens=lv_result.argmax("vocab")[{"batch": 0}], targets=lv_prompt["input_ids"][{"batch": 0}], mask_token_id=tokenizer.mask_token_id, attn_mask=lv_prompt["attention_mask"][{"batch": 0}])

In [38]:
from levanter.models.lm_model import MaskedLmExample

mask_prob = 0.15
mask_token_id = tokenizer.mask_token_id
noise_prob = 0.1

QPos = Pos
KPos = KeyPos

def _create_mlm_example(tokens, key):
    tokens_array = tokens.array
    targets = tokens_array.copy()

    if mask_prob > 0:
        this_key, key = jax.random.split(key)
        mask_shape = tokens_array.shape
        mask = jax.random.bernoulli(this_key, mask_prob, mask_shape)

        rand = jax.random.uniform(this_key, mask_shape)
        mask_token = jnp.where(rand < 0.8, mask_token_id, tokens_array)
        random_tokens = jax.random.randint(this_key, mask_shape, 0, tokens_array.max() + 1)
        mask_token = jnp.where((rand >= 0.8) & (rand < 0.8 + noise_prob), random_tokens, mask_token)
        masked_tokens = jnp.where(mask, mask_token, tokens_array)

        # Set targets to the original tokens where mask is True, otherwise set to mask_token_id
        targets = jnp.where(mask, tokens_array, mask_token_id)

        masked_tokens_named = hax.named(masked_tokens, QPos)
        targets_named = hax.named(targets, QPos)

        attn_mask_shape = (tokens_array.shape[0], tokens_array.shape[0])
        attn_mask = hax.named(jnp.ones(attn_mask_shape, dtype=jnp.bool_), (QPos, KPos))

        example = MaskedLmExample.masked_lm(tokens=masked_tokens_named, targets=targets_named, mask_token_id=mask_token_id, attn_mask=attn_mask)
    else:
        targets_named = hax.named(targets, QPos)
        attn_mask_shape = (tokens_array.shape[0], tokens_array.shape[0])
        attn_mask = hax.named(jnp.ones(attn_mask_shape, dtype=jnp.bool_), (QPos, KPos))

        example = MaskedLmExample.masked_lm(tokens=tokens, targets=targets_named, mask_token_id=mask_token_id, attn_mask=attn_mask)

    return example

example = _create_mlm_example(lv_prompt["input_ids"][{"batch": 0}], key_model)
lv_model.compute_loss(example, key=key_model, reduction=hax.mean, reduction_axis=Pos)

NamedArray(array=Array(15.134689, dtype=float32), axes=())

In [39]:
# check(named_to_tensor(example.tokens * 1.), named_to_tensor(lv_prompt["input_ids"]* 1.))

In [40]:
# tokenizer.decode(named_to_tensor(example.tokens))

In [41]:
# from haliax.nn import cross_entropy_loss
# import torch.types

# lv_logits = lv_model(example.tokens, example.attn_mask, position_ids=lv_prompt["position_ids"], key=key)
# lv_logits = lv_logits.astype(jnp.float32)

# hf_logits = hf_model(**{"input_ids" : named_to_tensor(example.tokens)[None, :], "attention_mask" : named_to_tensor(example.attn_mask)[None, :], "position_ids" : prompt["position_ids"]}).logits
# hf_logits = tensor_to_named(hf_logits.detach().numpy(), (Batch, Pos, Vocab))

# targets = example.targets
# target_y = hax.nn.one_hot(targets, Vocab, dtype=lv_logits.dtype)

# # jnp.dtype("long")

# lv_on_lv_loss = cross_entropy_loss(
#     lv_logits, Vocab, target_y, hax.mean, reduction_axis=Pos, where=example.loss_mask
# )

# hf_on_lv_loss = cross_entropy_loss(
#     hf_logits, Vocab, target_y, hax.mean, reduction_axis=Pos, where=example.loss_mask
# )

# # lv_loss = cross_entropy_loss(
# #     logits, Vocab, target_y, hax.mean, reduction_axis=Pos
# # )
# # target_y_torch = named_to_tensor(target_y)

# targets_torch = named_to_tensor(example.targets)
# # targets_torch = named_to_tensor(example.targets.astype(jnp.dtype("longlong")))

# targets_torch = targets_torch * named_to_tensor(example.loss_mask)
# targets_torch[targets_torch == 0] = -100
# targets_torch = torch.tensor(targets_torch, dtype=torch.long)

# lv_on_torch_loss = torch.nn.functional.cross_entropy(named_to_tensor(lv_logits)[0], targets_torch)
# hf_on_torch_loss = torch.nn.functional.cross_entropy(named_to_tensor(hf_logits)[0], targets_torch)

# print(lv_on_lv_loss)
# print(hf_on_lv_loss)
# print(lv_on_torch_loss)
# print(hf_on_torch_loss)

In [42]:
lv_result_logits.argmax(dim=-1)

tensor([[   0,    0, 1409,    5, 3497,    9, 1470,    4,    2,    2,    2,    2,
            2,    2,    2,    2,    2,    2,    2,    2,    2,    2,    2,    2,
            2,    2,    2,    2,    2,    2,    2,    2,    2,    2,    2,    2,
            2,    2,    2,    2,    2,    2,    2,    2,    2,    2,    2,    2,
            2,    2,    2,    2,    2,    2,    2,    2,    2,    2,    2,    2,
            2,    2,    2,    2,    2,    2,    2,    2,    2,    2,    2,    2,
            2,    2,    2,    2,    2,    2,    2,    2,    2,    2,    2,    2,
            2,    2,    2,    2,    2,    2,    2,    2,    2,    2,    2,    2,
            2,    2,    2,    2,    2,    2,    2,    2,    2,    2,    2,    2,
            2,    2,    2,    2,    2,    2,    2,    2,    2,    2,    2,    2,
            2,    2,    2,    2,    2,    2,    2,    2,    2,    2,    2,    2,
            2,    2,    2,    2,    2,    2,    2,    2,    2,    2,    2,    2,
            2,    2,    2,  

In [43]:
old_fn = ['</s>Markham is the city of California, located in San Diego, California</s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s>']

no_ids = ['</s>Markham is the city of California, located in San Diego, California</s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s>']

arange = ["</s>Mark</s> is the CEO of Facebook, located in San Angeles, California</s></s> cast cast cast cast cast cast cast cast cast cast cast cast cast cast cast cast cast cast cast cast cast cast cast cast cast cast cast cast cast</s> cast cast cast cast cast cast cast cast cast cast cast cast cast cast cast cast cast cast cast cast cast</s> cast</s></s></s></s></s></s></s></s> cast cast cast cast</s></s></s></s></s></s></s></s> cast</s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s> cast cast</s> cast cast cast</s></s></s></s></s></s> cast cast cast cast</s></s></s></s></s></s></s></s> cast</s> cast cast cast cast cast cast cast cast cast cast cast cast cast cast cast</s> cast cast</s></s></s> cast</s></s></s></s></s></s></s></s></s></s></s></s></s></s> cast cast</s></s></s></s></s></s> chance cast cast cast cast cast</s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s> cast cast cast cast cast cast</s></s></s></s></s></s></s></s> cast</s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s> cast cast cast cast cast cast cast cast cast cast cast cast cast cast cast cast cast cast cast cast cast cast cast cast cast cast cast cast cast cast cast cast cast cast cast cast cast cast cast cast cast cast cast cast cast cast cast cast cast cast cast</s> cast cast cast cast cast cast cast</s> cast cast cast cast cast cast cast cast cast cast cast cast cast cast cast cast cast cast cast cast cast cast</s></s></s></s></s> chance cast cast cast cast cast cast cast cast chance cast cast cast cast cast cast</s> cast</s></s></s></s></s></s></s></s></s></s></s></s></s></s> cast cast cast cast cast cast cast cast cast cast cast cast cast cast cast cast cast cast cast cast cast cast cast cast cast cast cast cast cast cast cast cast cast cast cast cast cast cast cast cast cast cast cast cast cast cast chance chance cast cast cast cast cast cast cast cast cast cast cast cast cast cast cast cast cast cast cast cast cast cast</s></s></s></s></s> cast cast cast cast cast cast cast cast cast cast cast cast cast cast cast cast cast</s></s></s></s></s></s></s></s></s> cast chance chance cast cast cast</s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s> chance cast</s></s></s></s></s></s></s></s></s></s></s></s></s></s></s> cast cast cast cast cast cast cast cast chance chance</s>"]

arange_no_offset = "nan"

old_fn_no_offset = ['<s><s>You are the age of America, located in the States, California</s></s> Crim Crim Crim Crim Crim Crim Crim Crim Crim Crim Crim Crim Crim Crim Crim Crim Crim Crim Crim Crim Crim Crim Crim Crim Crim Crim Crim Crim Crim Crim Crim Crim Crim Crim Crim Crim Crim Crim Crim Crim Crim Crim Crim Crim Crim Crim Crim Crim Crim Crim Crim Crim Crim Crim Crim Crim Crim Crim Crim Crim Crim Crim Crim Crim Crim Crim Crim Crim Crim Crim Crim Crim Crim Crim Crim Crim Crim Crim Crim Crim Crim Crim Crim Crim Crim Crim Crim Crim Crim Crim Crim Crim Crim Crim Crim Crim Crim Crim Crim Crim Crim Crim Crim Crim Crim Crim Crim Crim Crim Crim Crim Crim Crim Crim Crim Crim Crim Crim Crim Crim Crim Crim Crim Crim Crim Crim Crim Crim Crim Crim Crim Crim Crim Crim Crim Crim Crim Crim Crim Crim Crim Crim Crim Crim Crim Crim Crim Crim Crim Crim Crim Crim Crim Crim Crim Crim Crim Crim Crim Crim Crim Crim Crim Crim Crim Crim Crim Crim Crim Crim Crim Crim Crim Crim Crim Crim Crim Crim Crim Crim Crim Crim Crim Crim Crim Crim Crim Crim Crim Crim Crim Crim Crim Crim Crim Crim Crim Crim Crim Crim Crim Crim Crim Crim Crim Crim Crim Crim Crim Crim Crim Crim Crim Crim Crim Crim Crim Crim Crim Crim Crim Crim Crim Crim Crim Crim Crim Crim Crim Crim Crim Crim Crim Crim Crim Crim Crim Crim Crim Crim Crim Crim Crim Crim Crim Crim Crim Crim Crim Crim Crim Crim Crim Crim Crim Crim Crim Crim Crim Crim Crim Crim Crim Crim Crim Crim Crim Crim Crim Crim Crim Crim Crim Crim Crim Crim Crim Crim Crim Crim Crim Crim Crim Crim Crim Crim Crim Crim Crim Crim Crim Crim Crim Crim Crim Crim Crim Crim Crim Crim Crim Crim Crim Crim Crim Crim Crim Crim Crim Crim Crim Crim Crim Crim Crim Crim Crim Crim Crim Crim Crim Crim Crim Crim Crim Crim Crim Crim Crim Crim Crim Crim Crim Crim Crim Crim Crim Crim Crim Crim Crim Crim Crim Crim Crim Crim Crim Crim Crim Crim Crim Crim Crim Crim Crim Crim Crim Crim Crim Crim Crim Crim Crim Crim Crim Crim Crim Crim Crim Crim Crim Crim Crim Crim Crim Crim Crim Crim Crim Crim Crim Crim Crim Crim Crim Crim Crim Crim Crim Crim Crim Crim Crim Crim Crim Crim Crim Crim Crim Crim Crim Crim Crim Crim Crim Crim Crim Crim Crim Crim Crim Crim Crim Crim Crim Crim Crim Crim Crim Crim Crim Crim Crim Crim Crim Crim Crim Crim Crim Crim Crim Crim Crim Crim Crim Crim Crim Crim Crim Crim Crim Crim Crim Crim Crim Crim Crim Crim Crim Crim Crim Crim Crim Crim Crim Crim Crim Crim Crim Crim Crim Crim Crim Crim Crim Crim Crim Crim Crim Crim Crim Crim Crim Crim Crim Crim Crim Crim Crim Crim Crim Crim Crim Crim Crim Crim Crim Crim Crim Crim Crim Crim Crim Crim Crim Crim Crim']

In [44]:
old_fn_hf = ['<s>Mark Zuckerberg is the CEO of Facebook, located in San Alto, California.</s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s>']

no_ids_hf = ['<s>Mark Zuckerberg is the CEO of Facebook, located in San Alto, California.</s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s>']

arange_hf = ['<s></s><s>is the CEO of Facebook, located in San Francisco, California.</s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s>. Facebook.</s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s>']

arange_no_offset = "error"

old_fn_no_offset_hf = ['<s><s>Facebook is the CEO of Facebook, located in San Francisco, California.</s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s>']

In [45]:
print(lv_result_logits.shape)
print(hf_result_logits.shape)

torch.Size([1, 514, 50265])
torch.Size([1, 514, 50265])


In [46]:
tokenizer.batch_decode(lv_prompt["input_ids"].array, skip_special_tokens=False)

['<s>Paris is the<mask> of France.</s><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><p

In [47]:
tokenizer.batch_decode(prompt["input_ids"], skip_special_tokens=False)

['<s>Paris is the<mask> of France.</s><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><p

In [48]:
tokenizer.batch_decode(lv_result_logits.argmax(dim=-1), skip_special_tokens=False)

['<s><s>by the Republic of France.</s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></

In [49]:
tokenizer.batch_decode(hf_result_logits.argmax(dim=-1), skip_special_tokens=False)

['<s><s>by the Republic of France.</s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></

In [50]:
# from transformers import pipeline
# unmasker = pipeline('fill-mask', model='roberta-base')
# unmasker("The man worked as a <mask>.")

In [51]:
lv_result_logits

tensor([[[33.8544, -4.0704, 21.6643,  ...,  1.8860,  4.2439, 11.9759],
         [33.8159, -4.0739, 21.6653,  ...,  1.8778,  4.2321, 11.9650],
         [ 3.1134, -4.0773,  9.1893,  ..., -5.3634, -5.5656,  2.3893],
         ...,
         [12.4162, -4.4055, 29.7767,  ..., -1.2404, -4.1871,  8.8713],
         [12.4162, -4.4055, 29.7767,  ..., -1.2404, -4.1871,  8.8713],
         [12.4162, -4.4055, 29.7767,  ..., -1.2404, -4.1871,  8.8713]]])

In [52]:
hf_result_logits

tensor([[[33.8543, -4.0704, 21.6644,  ...,  1.8860,  4.2438, 11.9759],
         [33.8159, -4.0739, 21.6653,  ...,  1.8778,  4.2321, 11.9650],
         [ 3.1134, -4.0773,  9.1893,  ..., -5.3634, -5.5656,  2.3893],
         ...,
         [12.4163, -4.4055, 29.7767,  ..., -1.2404, -4.1871,  8.8713],
         [12.4163, -4.4055, 29.7767,  ..., -1.2404, -4.1871,  8.8713],
         [12.4163, -4.4055, 29.7767,  ..., -1.2404, -4.1871,  8.8713]]],
       grad_fn=<ViewBackward0>)

In [53]:
position_ids

NamedArray(array=Array([[1, 2, 3, 4, 5, 6, 7, 8, 9, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0

In [54]:
lv_result_logits.argmax(dim=-1)

tensor([[   0,    0, 1409,    5, 3497,    9, 1470,    4,    2,    2,    2,    2,
            2,    2,    2,    2,    2,    2,    2,    2,    2,    2,    2,    2,
            2,    2,    2,    2,    2,    2,    2,    2,    2,    2,    2,    2,
            2,    2,    2,    2,    2,    2,    2,    2,    2,    2,    2,    2,
            2,    2,    2,    2,    2,    2,    2,    2,    2,    2,    2,    2,
            2,    2,    2,    2,    2,    2,    2,    2,    2,    2,    2,    2,
            2,    2,    2,    2,    2,    2,    2,    2,    2,    2,    2,    2,
            2,    2,    2,    2,    2,    2,    2,    2,    2,    2,    2,    2,
            2,    2,    2,    2,    2,    2,    2,    2,    2,    2,    2,    2,
            2,    2,    2,    2,    2,    2,    2,    2,    2,    2,    2,    2,
            2,    2,    2,    2,    2,    2,    2,    2,    2,    2,    2,    2,
            2,    2,    2,    2,    2,    2,    2,    2,    2,    2,    2,    2,
            2,    2,    2,  

In [55]:
hf_result_logits.argmax(dim=-1)

tensor([[   0,    0, 1409,    5, 3497,    9, 1470,    4,    2,    2,    2,    2,
            2,    2,    2,    2,    2,    2,    2,    2,    2,    2,    2,    2,
            2,    2,    2,    2,    2,    2,    2,    2,    2,    2,    2,    2,
            2,    2,    2,    2,    2,    2,    2,    2,    2,    2,    2,    2,
            2,    2,    2,    2,    2,    2,    2,    2,    2,    2,    2,    2,
            2,    2,    2,    2,    2,    2,    2,    2,    2,    2,    2,    2,
            2,    2,    2,    2,    2,    2,    2,    2,    2,    2,    2,    2,
            2,    2,    2,    2,    2,    2,    2,    2,    2,    2,    2,    2,
            2,    2,    2,    2,    2,    2,    2,    2,    2,    2,    2,    2,
            2,    2,    2,    2,    2,    2,    2,    2,    2,    2,    2,    2,
            2,    2,    2,    2,    2,    2,    2,    2,    2,    2,    2,    2,
            2,    2,    2,    2,    2,    2,    2,    2,    2,    2,    2,    2,
            2,    2,    2,  

In [56]:
np.array(prompt["input_ids"])

array([[    0, 32826,    16,     5, 50264,     9,  1470,     4,     2,
            1,     1,     1,     1,     1,     1,     1,     1,     1,
            1,     1,     1,     1,     1,     1,     1,     1,     1,
            1,     1,     1,     1,     1,     1,     1,     1,     1,
            1,     1,     1,     1,     1,     1,     1,     1,     1,
            1,     1,     1,     1,     1,     1,     1,     1,     1,
            1,     1,     1,     1,     1,     1,     1,     1,     1,
            1,     1,     1,     1,     1,     1,     1,     1,     1,
            1,     1,     1,     1,     1,     1,     1,     1,     1,
            1,     1,     1,     1,     1,     1,     1,     1,     1,
            1,     1,     1,     1,     1,     1,     1,     1,     1,
            1,     1,     1,     1,     1,     1,     1,     1,     1,
            1,     1,     1,     1,     1,     1,     1,     1,     1,
            1,     1,     1,     1,     1,     1,     1,     1,     1,
      